[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nsiccha/hyperbolic_generic/blob/master/py/hydro_smooth_counterexample.ipynb)

In [12]:
import os
if os.getcwd() == '/content': #On colab, init
    !git clone https://github.com/nsiccha/hyperbolic_generic
    %cd /content/hyperbolic_generic/py/
!pip install cached_property sympy==1.6
from base import *
BasicExpr.math_env = 'aligned'
BasicExpr.show_label = False
from IPython.display import Markdown

import hydro

In [21]:
display(Markdown("# Jacobi Identity\nThese are the n.a.s. coditions for a hydrodynamic bracket to be Poisson:"))
display(hydro.jacobi_eqs)

# Jacobi Identity
These are the n.a.s. coditions for a hydrodynamic bracket to be Poisson:

Relations(Eq(0, ASymm(SupSub(MyTensor(g, 3, 0), (mu, i, j), ()), i, j)), Eq(0, MyAdd(PDiff(SupSub(MyTensor(g, 3, 0), (mu, i, j), ()), SupSub(MyTensor(u, 1, 0), (k,), ())), MyMul(-2, Symm(SupSub(MyTensor(b, 3, 1), (mu, i, j), (k,)), i, j)))), Eq(0, Symm(ASymm(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,))), i, j), mu, nu)), Eq(0, Cycler(ASymm(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,))), MyTuple(i, mu), MyTuple(j, nu)), i, j, k)), Eq(0, Symm(Protect(MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), ASymm(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), m, l)), Negative(ASymm(MyMul(SupSub(MyTensor(b, 3, 1), (nu, i, k), (l,)), SupSub(MyTensor(b, 3, 1), (mu, l, j), (m,))), j, k)))), mu, nu)), Eq(0, ASymm(Protect(MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (m,)), SupSub(MyTensor(u, 1, 0), (l,), ()))

In [23]:
mu_, i_, j_, k_, l_ = Wild.symbols('mu i j k l')
ansatzes = {
    g1[2,2]: 0, b1[2,2]: 0, b1[1,2]: 0
}, {
    g1[2,2]: -g1[1,1], b1[2,2]: -b1[1,1], b1[1,2]: b1[1,1], 
    b1[1,1].d(u[i_]): b1[1,1].d(u[1]), b1[1,1].d(u[i_], u[j_]): b1[1,1].d(u[1], u[1]),
}
display(Markdown('# Ansatzes\nThese are the two families of brackets from the proof.'))
display(Rep(ansatzes[0]))
display(Rep(ansatzes[1]))

# Ansatzes
These are the two families of brackets from the proof.

Rep({SupSub(MyTensor(b, 2, 1), (1, 2), ()): 0, SupSub(MyTensor(b, 2, 1), (2, 2), ()): 0, SupSub(MyTensor(g, 2, 0), (2, 2), ()): 0})

Rep({PDiff(SupSub(MyTensor(b, 2, 1), (1, 1), ()), SupSub(MyTensor(u, 1, 0), (i_,), ())): PDiff(SupSub(MyTensor(b, 2, 1), (1, 1), ()), SupSub(MyTensor(u, 1, 0), (1,), ())), PDiff(SupSub(MyTensor(b, 2, 1), (1, 1), ()), SupSub(MyTensor(u, 1, 0), (i_,), ()), SupSub(MyTensor(u, 1, 0), (j_,), ())): PDiff(SupSub(MyTensor(b, 2, 1), (1, 1), ()), SupSub(MyTensor(u, 1, 0), (1,), ()), SupSub(MyTensor(u, 1, 0), (1,), ())), SupSub(MyTensor(b, 2, 1), (1, 2), ()): SupSub(MyTensor(b, 2, 1), (1, 1), ()), SupSub(MyTensor(b, 2, 1), (2, 2), ()): Negative(SupSub(MyTensor(b, 2, 1), (1, 1), ())), SupSub(MyTensor(g, 2, 0), (2, 2), ()): Negative(SupSub(MyTensor(g, 2, 0), (1, 1), ()))})

In [19]:
nn = 2
for jj, ansatz in enumerate(ansatzes):
    display(Markdown(f"## Ansatz {jj+1}\nInserting the following ansatz:"))
    display(Rep(ansatz))
    for ii in range(len(hydro.jacobi_eqs.args)):
        display(Markdown(f"### Ansatz {jj+1}, Equation {ii+1}"))
        eq0 = hydro.jacobi_eqs.args[ii].rhs
        eq1 = eq0.doit().doit()
        eq2 = gMatrix(
            gSum(
                eq1,
                (l, nn),
                (n, nn)
            ),
            (i, nn),
            (j, nn),
            (k, nn),
            (m, nn)
        ) >> {
            g[mu_, i_, j_]: g1[i_, j_],
            b[mu_, i_, j_, k_]: b1[i_, j_, k_]
        } >> {
            g1[1,2]: 0, g1[2,1]: 0, b1[2,1,i_]:-b1[1,2,i_],
            g1[i_,i_].d(u[j_]): 2*b1[i_,i_,j_]
        } >> {
            b1[i_,j_,k_]: b1[i_,j_].d(u[k_])
        } << RRep(ansatz) >> {PDiff(i_, u[2], u[1]): PDiff(i_, u[1], u[2])} >> Distribute
        display(
            Eq(eq0, eq1, eq2)
        )


## Ansatz 1
Inserting the following ansatz:

Rep({SupSub(MyTensor(b, 2, 1), (1, 2), ()): 0, SupSub(MyTensor(b, 2, 1), (2, 2), ()): 0, SupSub(MyTensor(g, 2, 0), (2, 2), ()): 0})

### Ansatz 1, Equation 1

Eq(ASymm(SupSub(MyTensor(g, 3, 0), (mu, i, j), ()), i, j), MyMul(1/2, MyAdd(SupSub(MyTensor(g, 3, 0), (mu, i, j), ()), Negative(SupSub(MyTensor(g, 3, 0), (mu, j, i), ())))), MyMatrix((MyMatrix((MyMatrix((MyMatrix((0, 0)), MyMatrix((0, 0)))), MyMatrix((MyMatrix((0, 0)), MyMatrix((0, 0)))))), MyMatrix((MyMatrix((MyMatrix((0, 0)), MyMatrix((0, 0)))), MyMatrix((MyMatrix((0, 0)), MyMatrix((0, 0)))))))))

### Ansatz 1, Equation 2

...

Eq(Symm(Protect(MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), ASymm(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), m, l)), Negative(ASymm(MyMul(SupSub(MyTensor(b, 3, 1), (nu, i, k), (l,)), SupSub(MyTensor(b, 3, 1), (mu, l, j), (m,))), j, k)))), mu, nu), MyMul(1/2, MyAdd(Frac(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), MyAdd(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), Negative(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (m,)), SupSub(MyTensor(u, 1, 0), (l,), ()))))), 2), MyMul(-1/2, MyAdd(MyMul(SupSub(MyTensor(b, 3, 1), (nu, i, k), (l,)), SupSub(MyTensor(b, 3, 1), (mu, l, j), (m,))), MyMul(-1, SupSub(MyTensor(b, 3, 1), (nu, i, j), (l,)), SupSub(MyTensor(b, 3, 1), (mu, l, k), (m,))))), Frac(MyMul(SupSub(MyTensor(g, 3, 0), (nu, i, l), ()), MyAdd(PDiff(SupSub(MyTensor(b, 3, 1), (mu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), Negative(PDiff(SupSub(MyTensor(b, 3, 1), (mu, j, k), (m,)

### Ansatz 2, Equation 6

Eq(ASymm(Protect(MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (m,)), SupSub(MyTensor(u, 1, 0), (l,), ()))), MyMul(SupSub(MyTensor(b, 3, 1), (nu, j, i), (l,)), SupSub(MyTensor(b, 3, 1), (mu, l, k), (m,))), MyMul(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(b, 3, 1), (mu, i, l), (m,))))), MyTuple(i, mu), MyTuple(j, nu)), MyMul(1/2, MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (m,)), SupSub(MyTensor(u, 1, 0), (l,), ()))), MyMul(SupSub(MyTensor(b, 3, 1), (nu, j, i), (l,)), SupSub(MyTensor(b, 3, 1), (mu, l, k), (m,))), MyMul(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(b, 3, 1), (mu, i, l), (m,))), Negative(MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (nu, j, l), ()), PDiff(SupSub(MyTensor(b, 3, 1), (mu, i, k), (m,)), SupSub(MyTensor(u, 1, 0), (l,), ()))), MyMul(SupSub(MyTensor(b, 3, 1), (mu, i, j), (l,)), SupSub(MyTensor(b, 3, 1), (nu, l, k), (m,))), MyMul(Su

### Ansatz 2, Equation 7

Eq(Symm(Protect(MyAdd(PDiff(MyAdd(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), ASymm(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), m, l)), ASymm(MyMul(SupSub(MyTensor(b, 3, 1), (mu, i, j), (l,)), SupSub(MyTensor(b, 3, 1), (nu, l, k), (m,))), j, k)), SupSub(MyTensor(u, 1, 0), (n,), ())), Negative(Cycler(MyMul(SupSub(MyTensor(b, 3, 1), (mu, l, i), (m,)), ASymm(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), m, l)), i, j, k)))), MyTuple(mu, n), MyTuple(nu, l)), MyMul(1/2, MyAdd(PDiff(MyAdd(Frac(MyMul(SupSub(MyTensor(g, 3, 0), (mu, i, l), ()), MyAdd(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (l,)), SupSub(MyTensor(u, 1, 0), (m,), ())), Negative(PDiff(SupSub(MyTensor(b, 3, 1), (nu, j, k), (m,)), SupSub(MyTensor(u, 1, 0), (l,), ()))))), 2), MyMul(Frac(1, 2), MyAdd(MyMul(SupSub(MyTensor(b, 3, 1), (mu, i, j), (l,)), SupSub(MyTensor(b, 3, 1), (nu, l, k), (m,))), MyMul(-1, SupSub(MyTensor(b, 3, 1), (mu, i, k